In [1]:
# Installing the cassandra driver for python
!pip install cassandra-driver

     |████████████████████████████████| 4.1 MB 2.2 MB/s eta 0:00:01


In [1]:
import cassandra

Recheck the JDK path and direct to the bin folder of cassandra, the start cassandra with ./cassandra -f, once a local instance spins up, connect to the cluster

In [2]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [3]:
# Creating the keyspace (~ database)
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS alpha
        WITH REPLICATION =
            {
                'class' : 'SimpleStrategy',
                'replication_factor' : 1
            }
    """)
except Exception as e:
    print(e)

In [4]:
# Connect to the keyspace
try:
    session.set_keyspace('alpha')
except Exception as e:
    print(e)

Quereies I want to perform : 
select * from music_library where song_year >= 1970

song_year becomes a partition key!
artist_name is a clustering column.

In [5]:
# Usual commands
query = 'CREATE TABLE IF NOT EXISTS music_lib '

In [6]:
# Custom commands
query += '(year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))'

In [7]:
# Create table
try:
    session.execute(query)
except Exception as e:
    print(e)

In [8]:
# Running queries on the table
try:
    count = session.execute('SELECT COUNT(*) FROM MUSIC_LIB')
    print(count.all())
except Exception as e:
    print(e)

[Row(count=2)]


In [9]:
# Insert stuff into the table
query = 'insert into music_lib (year, artist_name, album_name) values (%s, %s, %s)'
try:
    session.execute(query, (2010, 'Linkin Park', 'A Thousand Suns'))
    session.execute(query, (2012, 'Linkin Park', 'Living Things'))
except Exception as e:
    print(e)

In [10]:
# Create table
try:
    rows = session.execute('SELECT * FROM MUSIC_LIB WHERE YEAR = 2010')
except Exception as e:
    print(e)

In [11]:
for row in rows:
    print(row.artist_name, row.year, row.album_name)

Linkin Park 2010 A Thousand Suns
